In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')
from sklearn.tree import DecisionTreeRegressor
import datetime as dt

In [17]:
df = pd.read_csv('../datos/cnt/11-cnt-cod_est.csv', index_col=0)
df.head(2)

,temp,atemp,hum,windspeed,season,holiday,weekday,weathersit,temp.1,cnt
0,-0.486274,-0.456399,0.855582,-0.208909,0,0,0,1,14.110847,985
1,-0.425690,-0.492863,0.332722,0.687550,0,2,0,1,14.902598,801


In [18]:
df.dtypes

temp          float64
atemp         float64
hum           float64
windspeed     float64
season          int64
holiday         int64
weekday         int64
weathersit      int64
temp.1        float64
cnt             int64
dtype: object

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        730 non-null    float64
 1   atemp       730 non-null    float64
 2   hum         730 non-null    float64
 3   windspeed   730 non-null    float64
 4   season      730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   weathersit  730 non-null    int64  
 8   temp.1      730 non-null    float64
 9   cnt         730 non-null    int64  
dtypes: float64(5), int64(5)
memory usage: 62.7 KB


In [20]:
df.head(2)

,temp,atemp,hum,windspeed,season,holiday,weekday,weathersit,temp.1,cnt
0,-0.486274,-0.456399,0.855582,-0.208909,0,0,0,1,14.110847,985
1,-0.425690,-0.492863,0.332722,0.687550,0,2,0,1,14.902598,801


---

In [21]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [23]:
arbol = DecisionTreeRegressor(random_state =0)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [24]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [25]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.0

In [26]:
print(arbol.tree_.max_depth)

24


In [27]:
param = {"max_depth": [2,4],
        "max_features": [1,2,3],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [28]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")
gs.fit(x_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [29]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                      min_samples_split=70)

In [30]:
#fig = plt.figure(figsize=(40, 20))
#tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [31]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [32]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [33]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "1_Dtree_CNT_est-cod")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,1161.578944,1.866076e+06,1366.043892,0.454817,test,1_Dtree_CNT_est-cod
1,1088.916498,1.613303e+06,1270.158691,0.577193,train,1_Dtree_CNT_est-cod


Random forest

In [34]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [35]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [36]:
param = {"max_depth": [2,4],
        "max_features": [1,2,3],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [37]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [38]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [39]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [40]:
'''for arbol in tqdm(bosque.estimators_):
    fig = plt.figure(figsize=(40, 20))
    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
    plt.show()'''

'for arbol in tqdm(bosque.estimators_):\n    fig = plt.figure(figsize=(40, 20))\n    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)\n    plt.show()'

In [41]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
   
   
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df


In [42]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "2_RForest_CNT_cod_estand")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,1070.724654,1.547600e+06,1244.025881,0.547861,test,2_RForest_CNT_cod_estand
1,1015.551926,1.420898e+06,1192.014386,0.627617,train,2_RForest_CNT_cod_estand


In [43]:
df_todos_resultados = pd.concat([dt_results, dt_results1], axis = 0)
df_todos_resultados


,MAE,MSE,RMSE,R2,set,modelo
0,1070.724654,1.547600e+06,1244.025881,0.547861,test,2_RForest_CNT_cod_estand
1,1015.551926,1.420898e+06,1192.014386,0.627617,train,2_RForest_CNT_cod_estand
0,1161.578944,1.866076e+06,1366.043892,0.454817,test,1_Dtree_CNT_est-cod
1,1088.916498,1.613303e+06,1270.158691,0.577193,train,1_Dtree_CNT_est-cod


In [44]:
df_todos_resultados.to_csv('../datos/cnt/12-cnt-metricas-estand.csv')

# Random Forest es la mejor opcion